<a href="https://colab.research.google.com/github/shahzaibparacha10-hash/Python_Class/blob/main/MINI_ACCOUNTING_ERP_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# Cell 1 — Setup (run this first every time)

from datetime import datetime

# 1) Define chart of accounts with simple types
ACCOUNTS = {
    "Cash": "Asset",
    "Bank": "Asset",
    "Accounts Receivable": "Asset",
    "Inventory": "Asset",
    "Accounts Payable": "Liability",
    "Capital": "Equity",
    "Sales": "Income",
    "Purchase": "Expense",
    "Rent Expense": "Expense",
    "Utilities Expense": "Expense",
}

# 2) Create the 'book' (balances + ledger lines)
# Each account keeps total debits, total credits, and a simple ledger list
book = {
    name: {
        "type": ACCOUNTS[name],
        "debits": 0.0,
        "credits": 0.0,
        "ledger": []  # each item: {"date": date, "narration": str, "debit": float, "credit": float}
    }
    for name in ACCOUNTS
}

def normal_side(account_type):
    # Assets & Expenses are debit-normal; others credit-normal
    return "debit" if account_type in ("Asset", "Expense") else "credit"

def account_balance(name):
    """Return balance respecting account's normal side."""
    rec = book[name]
    if normal_side(rec["type"]) == "debit":
        return rec["debits"] - rec["credits"]   # Dr balance
    else:
        return rec["credits"] - rec["debits"]   # Cr balance

def post_line(name, date, narration, debit=0.0, credit=0.0):
    """Post one ledger line and update totals (no validation here)."""
    rec = book[name]
    rec["debits"]  += float(debit)
    rec["credits"] += float(credit)
    rec["ledger"].append({
        "date": date,
        "narration": narration,
        "debit": round(float(debit), 2),
        "credit": round(float(credit), 2),
    })

print("Setup complete. Accounts loaded.")


Setup complete. Accounts loaded.


In [17]:
# Cell 2 — Show accounts

print("===== Chart of Accounts =====")
for name, rec in book.items():
    print(f"{name:<24}  ({rec['type']})")


===== Chart of Accounts =====
Cash                      (Asset)
Bank                      (Asset)
Accounts Receivable       (Asset)
Inventory                 (Asset)
Accounts Payable          (Liability)
Capital                   (Equity)
Sales                     (Income)
Purchase                  (Expense)
Rent Expense              (Expense)
Utilities Expense         (Expense)


In [ ]:
# Cell 3 — Post one simple journal (1 debit + 1 credit)

# Tip: use exact account names shown in Cell 2 (e.g., Cash, Sales, Rent Expense)

date_str = input("Date (YYYY-MM-DD) [Enter for today]: ").strip()
date = datetime.today().date() if date_str == "" else datetime.strptime(date_str, "%Y-%m-%d").date()

narration = input("Narration (e.g., Cash Sales): ").strip() or "General Entry"

debit_acc = input("Debit account: ").strip()
credit_acc = input("Credit account: ").strip()

if debit_acc not in book or credit_acc not in book:
    print("❌ Invalid account name. Re-run this cell and use names from the Chart of Accounts.")
else:
    try:
        amount = float(input("Amount (e.g., 15000): ").strip())
        if amount <= 0:
            print("❌ Amount must be positive.")
        else:
            # Post: Dr <debit_acc>, Cr <credit_acc>
            post_line(debit_acc, date, narration, debit=amount, credit=0.0)
            post_line(credit_acc, date, narration, debit=0.0, credit=amount)
            print(f"✅ Posted: {narration} | {date}")
            print(f"   Dr {debit_acc} {amount:.2f} | Cr {credit_acc} {amount:.2f}")
    except ValueError:
        print("❌ Invalid amount. Re-run and enter a number.")


In [ ]:
# Cell 4 — Trial Balance

print("========== Trial Balance ==========")
print(f"{'Account':<24} {'Debit':>12} {'Credit':>12}")
print("-"*50)

total_dr = total_cr = 0.0
for name, rec in book.items():
    bal = account_balance(name)

    # Show on normal side when positive (opposite if negative)
    if bal > 0:
        if normal_side(rec["type"]) == "debit":
            dr, cr = bal, 0.0
        else:
            dr, cr = 0.0, bal
    elif bal < 0:
        bal = abs(bal)
        if normal_side(rec["type"]) == "debit":
            dr, cr = 0.0, bal
        else:
            dr, cr = bal, 0.0
    else:
        dr = cr = 0.0

    if dr or cr:
        print(f"{name:<24} {dr:>12.2f} {cr:>12.2f}")
        total_dr += dr
        total_cr += cr

print("-"*50)
print(f"{'TOTAL':<24} {total_dr:>12.2f} {total_cr:>12.2f}")
print("✅ Balanced" if round(total_dr,2)==round(total_cr,2) else "❌ NOT Balanced")


In [ ]:
# Cell 5 — Ledger statement for one account

acc = input("Ledger — enter exact account name (e.g., Cash): ").strip()
if acc not in book:
    print("❌ Account not found. Re-run and type exactly as shown in Chart of Accounts.")
else:
    rec = book[acc]
    side = normal_side(rec["type"])
    lines = sorted(rec["ledger"], key=lambda x: x["date"])

    print(f"\n===== Ledger: {acc} ({rec['type']}) =====")
    print(f"{'Date':<12} {'Narration':<28} {'Debit':>10} {'Credit':>10} {'Balance':>12}")
    print("-"*75)

    running = 0.0
    for l in lines:
        if side == "debit":
            running += (l["debit"] - l["credit"])
            bal_val = abs(running)
            bal_side = "Dr" if running >= 0 else "Cr"
        else:
            running += (l["credit"] - l["debit"])
            bal_val = abs(running)
            bal_side = "Cr" if running >= 0 else "Dr"

        print(f"{l['date']}  {l['narration'][:27]:<28} {l['debit']:>10.2f} {l['credit']:>10.2f} {bal_val:>9.2f} {bal_side}")


In [ ]:
# Cell 6 — Profit & Loss

income = expense = 0.0
for name, rec in book.items():
    if rec["type"] == "Income":
        income += (rec["credits"] - rec["debits"])   # credit-normal
    elif rec["type"] == "Expense":
        expense += (rec["debits"] - rec["credits"])  # debit-normal

net = income - expense

print("\n========== Profit & Loss ==========")
print(f"Total Income : {income:>12.2f}")
print(f"Total Expense: {expense:>12.2f}")
print("-"*30)
if net >= 0:
    print(f"Net Profit   : {net:>12.2f} (Credit → adds to Equity)")
else:
    print(f"Net Loss     : {abs(net):>12.2f} (Debit  → reduces Equity)")


In [ ]:
# Cell 7 — Balance Sheet (includes current period P&L as retained earnings effect)

# 1) Compute P&L for retained earnings
income = expense = 0.0
for name, rec in book.items():
    if rec["type"] == "Income":
        income += (rec["credits"] - rec["debits"])
    elif rec["type"] == "Expense":
        expense += (rec["debits"] - rec["credits"])
net_pl = income - expense  # profit (+equity) / loss (-equity)

# 2) Sum balances by type
assets = liabilities = equity = 0.0
for name, rec in book.items():
    bal = account_balance(name)
    if rec["type"] == "Asset":
        assets += bal
    elif rec["type"] == "Liability":
        liabilities += bal
    elif rec["type"] == "Equity":
        equity += bal

equity_total = equity + net_pl  # P&L flows into equity
rhs = liabilities + equity_total

print("\n========== Balance Sheet ==========")
print(f"Total Assets                 : {assets:>12.2f}")
print(f"Total Liabilities            : {liabilities:>12.2f}")
print(f"Equity (incl. Ret. Earnings) : {equity_total:>12.2f}")
print("-"*40)
print(f"LHS (Assets)                 : {assets:>12.2f}")
print(f"RHS (Liab + Equity)          : {rhs:>12.2f}")
print("✅ Balances" if round(assets,2)==round(rhs,2) else "❌ Does NOT balance")


In [ ]:
# Cell 8 — Seed demo data (no typing)

today = datetime.today().date()

# 1) Opening Capital: Cash Dr 100,000 / Capital Cr 100,000
post_line("Cash", today, "Opening Capital", debit=100000, credit=0)
post_line("Capital", today, "Opening Capital", debit=0, credit=100000)

# 2) Inventory purchase (cash): Inventory Dr 20,000 / Cash Cr 20,000
post_line("Inventory", today, "Inventory (cash)", debit=20000, credit=0)
post_line("Cash", today, "Inventory (cash)", debit=0, credit=20000)

# 3) Cash sales: Cash Dr 15,000 / Sales Cr 15,000
post_line("Cash", today, "Cash Sales", debit=15000, credit=0)
post_line("Sales", today, "Cash Sales", debit=0, credit=15000)

# 4) Pay rent: Rent Expense Dr 8,000 / Cash Cr 8,000
post_line("Rent Expense", today, "Monthly Rent", debit=8000, credit=0)
post_line("Cash", today, "Monthly Rent", debit=0, credit=8000)

print("✅ Demo transactions posted. Now run:")
print("   • Cell 4 (Trial Balance)")
print("   • Cell 5 (Ledger) — try Cash, Sales, Rent Expense")
print("   • Cell 6 (Profit & Loss)")
print("   • Cell 7 (Balance Sheet)")
